**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.core.display import HTML
import cv2
print (cv2.__version__)
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices, color = 255):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (color,) * channel_count
    else:
        ignore_mask_color = color
        
            
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def hough_lines_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an set with detected lines
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def hough_lines_rhoTheta(img, rho, theta, threshold):
    return cv2.HoughLines(image, rho, theta, threshold)

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

# images
- read all images from "test_images"
- create a list of images: **images**

In [ ]:
import os

# read images from a dir
# presumption: the dir has images only
# return: a list of [image, "name_of_the_image"] items
def readImagesFromDir(dir):
    names =  os.listdir("test_images/")
    images = []
    for name in names:
        images.append(mpimg.imread('test_images/' + name))
    return images
    

In [ ]:
def showImages(list, columns, rows = 1, isGray = False):
    fig = plt.figure(figsize=(columns * 5, rows * 5))
    i = 1
    print("columns=", columns, " rows=", rows, " isGray=", isGray)
    for img in list:
        fig.add_subplot(rows, columns, i)
        i = i + 1 
        if(isGray):
            plt.imshow(img, cmap='gray')
        else:
            plt.imshow(img)
        
    plt.show()  # finally, render the plot

In [ ]:
## How to use subplots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
images = readImagesFromDir("/test_images/");

In [ ]:
#showImages(images, len(images))
showImages(images, 6,1)

# imgRegion
- pts: point set for creation the the region of interest
- imgRegion

## define point sets for masking and clipping
- Outer polygon - define the region of interest
- Inner polygon - clip the the "Outer polygon"-edges - detected by the Canny-Algorithm¶

In [ ]:
dN=10 
yMax = 540
xMax = 960
y2   = 320

# point set for images
p1x  = 110
p1y  = yMax
p2x  = 390
p2y  = y2
p3x  = 550
p3y  = y2
p4x  = 940
p4y  = yMax
pts       = np.array([(p1x, p1y),(p2x, p2y),(p3x, p3y),(p4x, p4y)], np.int32)
pts_clipp = np.array([(p1x+dN, p1y),(p2x+dN, p2y+dN),(p3x-dN, p3y+dN),(p4x-dN, p4y)], np.int32)



In [ ]:
# point set for videos
pv1x  = 100
pv1y  = yMax
pv2x  = 420
pv2y  = y2
pv3x  = 550
pv3y  = y2
pv4x  = 930
pv4y  = yMax

pts_video       = np.array([(pv1x, pv1y),(pv2x, pv2y),(pv3x, pv3y),(pv4x, pv4y)], np.int32)
pts_clipp_video = np.array([(pv1x+dN, pv1y),(pv2x+dN, pv2y+dN),(pv3x-dN, pv3y+dN),(pv4x-dN, pv4y)], np.int32)

In [ ]:
print((image.shape[0], image.shape[1], image.shape[2]))

In [ ]:
dim = xMax * yMax * 3
imgWhite = np.zeros(dim, np.uint8)
imgWhite += 255;
imgWhite = imgWhite.reshape(yMax, xMax, 3)
imgMask = region_of_interest(imgWhite, [pts])
print("imgMask.shape()",imgMask.shape)
plt.imshow(imgMask)

In [ ]:
pts, ps_clipp = getPointSets()

In [ ]:
def  houghCannyChain(image):
    imgRegion       = region_of_interest(image, [pts_video])
    imgGray         = grayscale(imgRegion)
    kernel = 5
    imgGaussBlur    = gaussian_blur(imgGray, kernel) 
    lowTh = 100
    highTh = 200
    imgCanny        = canny(imgGaussBlur, lowTh, highTh)
    imgCannyClipped = region_of_interest(imgCanny, [pts_clipp_video])
    linesHough      = hough_lines_lines(imgCannyClipped, 1, np.pi/180, 5, 5,2) 
    imgOrgAndHough  = np.copy(image)
    draw_lines(imgOrgAndHough, linesHough)
    # cv2.line(imgOrgAndHough, (pv1x, pv1y), (pv2x, pv2y), [0,255,0], 4)
    # cv2.line(imgOrgAndHough, (pv3x, pv3y), (pv4x, pv4y), [0,255,0], 4)
    return imgOrgAndHough

In [ ]:
i1 = houghCannyChain(image)

In [ ]:
plt.imshow(i1)

In [ ]:
# attention: pts only doesnt work - [pts] is necessary!
imgRegions = [region_of_interest(image, [pts]) for image in images]
showImages(imgRegions, len(imgRegions))

In [ ]:
plt.imshow(imgRegions[0])

# imgMask

In [ ]:
dim = image.shape[0] * image.shape[1] * image.shape[2]
print("dim=",dim)
imgWhite = np.zeros(dim, np.uint8)
imgWhite += 255;
imgWhite = imgWhite.reshape(image.shape[0], image.shape[1], image.shape[2])
print("imgZeros.shape", imgWhite.shape)
print("imgWhite[0,0]", imgWhite[0,0])

imgMask = region_of_interest(imgWhite, [pts])
print("imgMask.shape()",imgMask.shape)
plt.imshow(imgMask)




# imgGray

In [ ]:
imgGrays = [grayscale(image) for image in imgRegions]
showImages(imgGrays, len(imgGrays),1, True)

# imgGaussianBlur

In [ ]:
kernel = 3
imgGaussianBlurs_3 = [gaussian_blur(image, kernel) for image in imgGrays]
showImages(imgGaussianBlurs_3, len(imgGaussianBlurs_3),1, True)

In [ ]:
kernel = 5
imgGaussianBlurs_5 = [gaussian_blur(image, kernel) for image in imgGrays]
showImages(imgGaussianBlurs_5, len(imgGaussianBlurs_5),1, True)

In [ ]:
kernel = 5
imgGaussianBlurs = [gaussian_blur(image, kernel) for image in imgGrays]
showImages(imgGaussianBlurs, len(imgGaussianBlurs),1, True)

## select parameter for GaussianBlur 
- kernel = 5

In [ ]:
low_thresholds  =[80,100,120]
high_thresholds = [160,200,240]

def cannyList(lowTh, highTh):
    return [canny(image, lowTh, highTh) for image in imgGaussianBlurs ]

imgCannysSum = []
for lowTh in low_thresholds:
    for highTh in high_thresholds:
        imgCannysSum += cannyList(lowTh, highTh)

showImages(imgCannysSum, 6, 9, True)

In [ ]:
kernel = 5
imgGaussianBlurs = [gaussian_blur(image, kernel) for image in imgGrays]
showImages(imgGaussianBlurs, len(imgGaussianBlurs),1, True)

# imgCanny

## select parameter for Canny
- lowThreshold = 100
- highThreshold = 200

In [ ]:
lowTh = 100
hithTh = 200
imgCanny = cannyList(lowTh, highTh)
showImages(imgCanny, 6, 1, True)

# delete the "region-edge" from the edge-detection
- The area for edge detection was restricted to the region "of interest".
- This was realized by a masking.
- The border of the mask was detect as a line too. But This edges are not realy eges of the picture - and will be removed.

In [ ]:
imgCannysNoExternalEdges = [region_of_interest(image, [pts_clipp]) for image in imgCanny]
showImages(imgCannysNoExternalEdges, 6, 1, True)

# imgHough

In [ ]:
imgRhoTeta = [cv2.HoughLines(image, 1, np.pi/180, 5) for image in imgCannysNoExternalEdges]
# showImages(imgRhoTeta, 3, 2, True)


In [ ]:
print(len(imgRhoTeta[0]))

In [ ]:
#( dst, lines, 1, CV_PI/360, 8, 0, 0 );
# imgHough = hough_lines(imgCanny, 1, np.pi/180, 0, 5, 1)
# plt.imshow(imgHough, cmap='gray') 


imgHoughs = [hough_lines(image, 1, np.pi/180, 5, 5,2) for image in imgCannysNoExternalEdges]
showImages(imgHoughs, 3, 2, True)

In [ ]:
linesHough = [hough_lines_lines(image, 1, np.pi/180, 5, 5,2) for image in imgCannysNoExternalEdges]

# Merge orgImages and imgHoughs -> save


In [ ]:
# copy (save) the original images
imgOrgAndHoughs = [np.copy(image) for image in images]
# draw_lines(line_img, lines)
for i in range(len(imgOrgAndHoughs)):
    draw_lines(imgOrgAndHoughs[i], linesHough[i])
# map(function_to_apply, list_of_inputs) # more as one aragument
showImages(imgOrgAndHoughs, 3, 2, True)


In [ ]:
plt.imshow(imgOrgAndHoughs[5])

## Save the images
- src: imgOrgAndHoughs (what)
- dest: \test_images_output\
- name: "org"_out.jpg

In [ ]:
from pathlib import Path
def writeImagesToDir(dir_, images):
    names_with_ext =  os.listdir("test_images/")
    names = [Path(name).stem for name in names_with_ext]
            
    for i in range(len(images)):
        fileName = dir_ + "/"+names[i] + "_out.jpg"
        cv2.imwrite(fileName, images[i])    

    

writeImagesToDir('test_images_output', imgOrgAndHoughs)

Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

# Filter (discriminate) lines regarding the left/right side.
- Discrimination criteria: gradient

In [ ]:
### lineParametersLines: 

In [ ]:
lineParameterLines = []
for lines in linesHough:
    lineParameters = []
    teta_avg = 0
    count = 0
    for l in lines:
        line = l[0]
        # yMax, len,  m, n
        dx = line[2] - line[0]
        dy = line[3] - line[1]
        if(dy != 0):
            length = math.sqrt(dx*dx + dy*dy)
            teta = dx/dy # ÄNDERN
            n = line[1] - m * line[0]
            #print(m,n,length)
 #           print("% 10.1E % 10.1E % d "% (m,n,length))
            teta_avg += teta
            count += 1
            yMax = max(line[1], line[3])
            lineParameters.append([yMax, length, m, n])
    lineParameterLines.append([m_teta/count,lineParameters])
#    print("---------")
# print((lineParameterLines[0][1][2]))
# print(lineParameterLines[0][0])


In [ ]:
len(lineParameterLines)


In [ ]:
def separeteLaneSites (lineParameterLin):
    left = []
    right = []
    

## Build a Lane Finding Pipeline



In [ ]:
# all pictures have the same dimensions. Read the y size
# motivation: later the y-coordinate is interpreted as high or distance and is a paramter to rate the line
# denpends on the high and length the current line is more or less important for calculate the average slope for the lane decorator 
imgMaxY = imgOrgAndHoughs[0].shape[1]

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# imageio.plugins.ffmpeg.download()

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)

    return houghCannyChain(image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
display(HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output)))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

In [ ]:
import cv2 as cv
import numpy as np
W = 400
def my_ellipse(img, angle):
    thickness = 2
    line_type = 8
    cv.ellipse(img,
                (W // 2, W // 2),
                (W // 4, W // 16),
                angle,
                0,
                360,
                (255, 0, 0),
                thickness,
                line_type)
def my_filled_circle(img, center):
    thickness = -1
    line_type = 8
    cv.circle(img,
               center,
               W // 32,
               (0, 0, 255),
               thickness,
               line_type)
def my_polygon(img):
    line_type = 8
    # Create some points
    ppt = np.array([[W / 4, 7 * W / 8], [3 * W / 4, 7 * W / 8],
                    [3 * W / 4, 13 * W / 16], [11 * W / 16, 13 * W / 16],
                    [19 * W / 32, 3 * W / 8], [3 * W / 4, 3 * W / 8],
                    [3 * W / 4, W / 8], [26 * W / 40, W / 8],
                    [26 * W / 40, W / 4], [22 * W / 40, W / 4],
                    [22 * W / 40, W / 8], [18 * W / 40, W / 8],
                    [18 * W / 40, W / 4], [14 * W / 40, W / 4],
                    [14 * W / 40, W / 8], [W / 4, W / 8],
                    [W / 4, 3 * W / 8], [13 * W / 32, 3 * W / 8],
                    [5 * W / 16, 13 * W / 16], [W / 4, 13 * W / 16]], np.int32)
    ppt = ppt.reshape((-1, 1, 2))
    cv.fillPoly(img, [ppt], (255, 255, 255), line_type)
    # Only drawind the lines would be:
    # cv.polylines(img, [ppt], True, (255, 0, 255), line_type)
def my_line(img, start, end):
    thickness = 2
    line_type = 8
    cv.line(img,
             start,
             end,
             (0, 0, 0),
             thickness,
             line_type)
atom_window = "Drawing 1: Atom"
rook_window = "Drawing 2: Rook"
# Create black empty images
size = W, W, 3
atom_image = np.zeros(size, dtype=np.uint8)
rook_image = np.zeros(size, dtype=np.uint8)
# 1.a. Creating ellipses
my_ellipse(atom_image, 90)
my_ellipse(atom_image, 0)
my_ellipse(atom_image, 45)
my_ellipse(atom_image, -45)
# 1.b. Creating circles
my_filled_circle(atom_image, (W // 2, W // 2))
# 2. Draw a rook
# ------------------
# 2.a. Create a convex polygon
my_polygon(rook_image)
cv.rectangle(rook_image,
              (0, 7 * W // 8),
              (W, W),
              (0, 255, 255),
              -1,
              8)
#  2.c. Create a few lines
my_line(rook_image, (0, 15 * W // 16), (W, 15 * W // 16))
my_line(rook_image, (W // 4, 7 * W // 8), (W // 4, W))
my_line(rook_image, (W // 2, 7 * W // 8), (W // 2, W))
my_line(rook_image, (3 * W // 4, 7 * W // 8), (3 * W // 4, W))
cv.imshow(atom_window, atom_image)
cv.moveWindow(atom_window, 0, 200)
cv.imshow(rook_window, rook_image)
cv.moveWindow(rook_window, W, 200)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
import numpy as np
import cv2

# Create a black image
img = np.zeros((512,512,3), np.uint8)

fig=plt.figure()
rows = 3
collumns = 2

fig.add_subplot(rows, collumns, 1)
plt.imshow(img)

# Draw a diagonal blue line with thickness of 5 px
imgLine = cv2.line(img,(0,0),(511,511),(255,0,0),5)
fig.add_subplot(rows, collumns, 2)
plt.imshow(imgLine)
img = np.zeros((512,512,3), np.uint8)

# plt.imshow(img)
imgRectangle = cv2.rectangle(img,(384,0),(510,128),(0,255,0),3)
fig.add_subplot(rows, collumns, 3)
plt.imshow(imgRectangle)
img = np.zeros((512,512,3), np.uint8)

imgEllipse = cv2.ellipse(img,(256,256),(100,50),0,0,180,255,-1)
fig.add_subplot(rows, collumns, 4)
plt.imshow(imgEllipse)
img = np.zeros((512,512,3), np.uint8)

pts = np.array([[10,5],[20,30],[70,20],[50,10]], np.int32)
pts = pts.reshape((-1,1,2))
imgPoly = cv2.polylines(img,[pts],True,(0,255,255))
fig.add_subplot(rows, collumns, 5)
plt.imshow(imgPoly)
img = np.zeros((512,512,3), np.uint8)



plt.show()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Create a black image
img = np.zeros((512,512,3), np.uint8)

pts = np.array([[10,5],[20,30],[70,20],[50,10]], np.int32)
print(pts)
print(pts.shape)
pts = pts.reshape((-1,1,2))
print(pts)
print(pts.shape)

imgPoly = cv2.polylines(img,[pts],True,(0,255,255))
plt.imshow(imgPoly) 

In [ ]:
test = np.zeros((2,4,3), np.int8)

for i in range(2):
    for j in range(4):
        for k in range(3):
            test[i,j,k]=i*2 + j*3 +k*10 
print(test)    
plt.imshow(test) 

In [ ]:
red_threshold = 0
green_threshold = 0
blue_threshold = 0
rgb_threshold = [red_threshold, green_threshold, blue_threshold]
color_thresholds = (test[:,:,0] < rgb_threshold[0])
test2 = test[:,:,0]
print(color_thresholds)
print(test2)